In [1]:
from typing import TypedDict, cast
from prisma import Prisma
from prisma.fields import Json

In [3]:
db = Prisma(auto_register=True)
await db.connect()

INSERT / UPDATE 時には dict を入れることができるが、型を Json にしないといけない

In [ ]:
await db.parent.create(
    {
        "name": "foo",
        "meta": Json({}),
        "children": {
            "create": [
                {
                    "name": "bar",
                    "meta": Json({}),
                }
            ]
        },
    }
)

Parent(id='6db14242-917f-40f6-86f7-cd07979d268e', created_at=datetime.datetime(2025, 5, 12, 9, 36, 44, 413000, tzinfo=TzInfo(UTC)), children=None, name='foo', meta={})

値がただの dict ではなく TypedDict の場合は「Serializable でない」として型エラーが出るが、dict に cast すれば問題ない

In [4]:
class ParentMeta(TypedDict):
    count: int


class ChildMeta(TypedDict):
    count: int


parent_meta: ParentMeta = {"count": 123}
child_meta: ChildMeta = {"count": 456}


await db.parent.create(
    {
        "name": "foo",
        "meta": Json(cast(dict, parent_meta)),
        "children": {
            "create": [
                {
                    "name": "bar",
                    "meta": Json(cast(dict, child_meta)),
                }
            ]
        },
    }
)

Parent(id='38fb9419-595d-41fc-97cf-331f44a80260', created_at=datetime.datetime(2025, 5, 12, 9, 41, 57, 609000, tzinfo=TzInfo(UTC)), children=None, name='foo', meta={'count': 123})

SELECT: 実態は dict だが型ヒントは Json となっている (Annotated なので問題ない？)

In [5]:
parent = await db.parent.find_first_or_raise(
    order={"created_at": "desc"},
    include={"children": True},
)
print(type(parent.meta))  # 型ヒントは Json 型

<class 'dict'>
